# MindAlpha Getting Started

MindAlpha is a machine learning platform, which provides a one-stop solution for data preprocessing, model training and online prediction.

In this article, we introduce the basic API of MindAlpha briefly. A more complete example can be found in [MindAlpha Tutorial](mindalpha-tutorial.ipynb).

## Prepare Data

We use the publicly available dataset [Terabyte Click Logs](https://labs.criteo.com/2013/12/download-terabyte-click-logs-2/) published by CriteoLabs as our demo dataset.

We sample the dataset with sampling rate 0.001 so that the running of the demo can finish quickly. More information about the demo dataset can be found in [MindAlpha Demo Dataset](https://mob-emr-test.s3.amazonaws.com/ml-platform/ml-ranking/data/criteo/0.001/index.html).

Execute the following cell to download the demo dataset into the working directory. Those data files take up about 2.1 GiB disk space and the downloading process may take sveral minutes. If the downloading fails, please refer to [MindAlpha Demo Dataset](https://mob-emr-test.s3.amazonaws.com/ml-platform/ml-ranking/data/criteo/0.001/index.html) and download the dataset manually.

In [ ]:
import mindalpha
mindalpha.demo.download_dataset()

You can check the downloaded dataset by executing the following cell.

In [ ]:
!ls -l ${PWD}/data/

To upload the dataset to your own s3 bucket:

1. Fill ``{YOUR_S3_BUCKET}`` and ``{YOUR_S3_PATH}`` with your preferred values in the following cell.
2. Uncomment the cell by removing the leading ``#`` character.
3. Execute the cell.

In [ ]:
#!aws s3 cp --recursive ${PWD}/data/ s3://{YOUR_S3_BUCKET}/{YOUR_S3_PATH}/demo/data/

Alternatively, you can open a terminal by selecting the ``File`` -> ``New`` -> ``Terminal`` menu item and executing Bash commands in it.

You can check the uploaded dataset in your s3 bucket by uncommenting and executing the following cell.

In [ ]:
#!aws s3 ls s3://{YOUR_S3_BUCKET}/{YOUR_S3_PATH}/demo/data/

## Define the Model

We can define our neural network model by subclassing ``torch.nn.Module`` as usual PyTorch models. The following ``DemoModule`` class provides an example.

Compared to usual PyTorch models, the notable difference is the ``_sparse`` layer created by instantiating ``ma.EmbeddingSumConcat`` which takes an embedding size and paths of two text files. ``ma.EmbeddingSumConcat`` makes it possible to define large-scale sparse models in PyTorch, which is a distinguishing feature of MindAlpha.

In [ ]:
import torch
import mindalpha as ma

class DemoModule(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self._embedding_size = 16
        self._schema_dir = './schema/'
        self._column_name_path = self._schema_dir + 'column_name_demo.txt'
        self._combine_schema_path = self._schema_dir + 'combine_schema_demo.txt'
        self._sparse = ma.EmbeddingSumConcat(self._embedding_size, self._column_name_path, self._combine_schema_path)
        self._sparse.updater = ma.FTRLTensorUpdater()
        self._sparse.initializer = ma.NormalTensorInitializer(var=0.01)
        self._dense = torch.nn.Sequential(
            ma.nn.Normalization(self._sparse.feature_count * self._embedding_size),
            torch.nn.Linear(self._sparse.feature_count * self._embedding_size, 1024),
            torch.nn.ReLU(),
            torch.nn.Linear(1024, 512),
            torch.nn.ReLU(),
            torch.nn.Linear(512, 1),
        )

    def forward(self, x):
        x = self._sparse(x)
        x = self._dense(x)
        return torch.sigmoid(x)

Instantiating the ``DemoModule`` class to define our PyTorch model.

In [ ]:
module = DemoModule()

## Train the Model

To train our model, first we need to create a ``ma.PyTorchEstimator`` passing in several arguments including our PyTorch model ``module`` and the number of workers and servers.

``model_out_path`` specifies where to store the trained model, we use a local directory path here.

``input_label_column_index`` specifies the column index of the label column in the dataset, which is ``0`` for the demo dataset.

In [ ]:
model_out_path = './output/model_out/'
estimator = ma.PyTorchEstimator(module=module,
                                worker_count=1,
                                server_count=1,
                                model_out_path=model_out_path,
                                input_label_column_index=0)

Next, we create a Spark session by calling ``ma.spark.get_session()`` and load the training dataset by call ``ma.input.read_s3_csv()``.

``delimiter`` specifies the column delimiter of the dataset, which is the TAB character ``'\t'`` for the demo dataset.

In [ ]:
train_dataset_path = './data/train/day_0_0.001_train.csv'
spark_session = ma.spark.get_session(local=True,
                                     batch_size=100,
                                     worker_count=estimator.worker_count,
                                     server_count=estimator.server_count)
train_dataset = ma.input.read_s3_csv(spark_session, train_dataset_path, delimiter='\t')

Finally, we call the ``fit()`` method of ``ma.PyTorchEstimator`` to train our model. This will take several minutes and you can see the progress by looking at the output of the cell. The trained model is stored in ``model_out_path`` and the ``model`` variable.

In [ ]:
model = estimator.fit(train_dataset)

## Evaluate the Model

To evaluate our model, we use the ``ma.input.read_s3_csv()`` function again to load the test dataset, passing in the column delimiter ``'\t'``.

In [ ]:
test_dataset_path = './data/test/day_0_0.001_test.csv'
test_dataset = ma.input.read_s3_csv(spark_session, test_dataset_path, delimiter='\t')

Next, we call the ``model.transform()`` method to transform the test dataset, which will add a column named ``rawPrediction`` to the test dataset representing the predicted labels. For ease of integration with Spark ML, ``model.transform()`` will also add a column named ``label`` to the test dataset representing the actual labels.

Like the training process, this will take several minutes and you can see the progress by looking at the output of the cell. The transformed test dataset is stored in the ``result`` variable.

In [ ]:
result = model.transform(test_dataset)

``result`` is a normal PySpark DataFrame and can be inspected by its methods.

In [ ]:
result.show(5)

Finally, we use ``pyspark.ml.evaluation.BinaryClassificationEvaluator`` to compute test AUC.

In [ ]:
import pyspark
evaluator = pyspark.ml.evaluation.BinaryClassificationEvaluator()
test_auc = evaluator.evaluate(result)
print('test_auc: %g' % test_auc)

## Summary

We illustrated how to train and evaluate neural network model in MindAlpha. Users familiar with PyTorch and Spark ML should get started easily, which is the design goal of MindAlpha.

A more complete example can be found in [MindAlpha Tutorial](mindalpha-tutorial.ipynb), which illustrats how MindAlpha is used in the production environment.